In [1]:
import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.graph_objs as go

In [3]:
from datetime import datetime as dt
from iexfinance.stocks import get_historical_data

In [4]:
import pickle

In [5]:
# Get tickers (previously downloaded from NASDAQ site)
with open("symbols.pkl", "rb") as t:
    tickers = pickle.load(t) 

In [6]:
from iexfinance.stocks import Stock

In [1]:
# Get the company name associated with every ticker from the iexfinance API (only needs to be run once)
i = 0
j = 50
names = {}


for i in range(0,len(tickers),50):
    stock_info = Stock(tickers[i:j],token="xxxxxxxx")
    names.update(stock_info.get_company_name())
    j = j + 50

In [ ]:
# Setting app layout
app = dash.Dash()

app.layout = html.Div([
    html.Div([html.H1('Stock Ticker Dashboard')]
            ),
    html.Div([html.Div([dcc.Dropdown(id='ticker-dd',
                          options=[{
                           'label': name + ', ' + names.get(name), 
                           'value': name} for name in names],
                           value = list(names.keys())[0],
                           multi=True
                           )],style={'display':'inline-block','width':'30%','vertical-align':'top'}),
    html.Div([dcc.DatePickerRange(id='date',
                                  min_date_allowed=dt(2000, 1, 1),
                                  max_date_allowed=dt(2020, 3, 19),
                                  initial_visible_month=dt(2019, 1, 1),
                                  start_date=dt(2020, 1, 1),
                                  end_date=dt(2020, 3, 19))],
             style={'display':'inline-block','width':'30%','padding':'5px'}),
    html.Div([html.Button('Submit', id='button', type='submit')],
             style={'display':'inline-block','width':'30%','padding':'5px'})]),
    html.Div([dcc.Graph(id='price-chart',
                       ),
    html.Div(id='output')
             ])
])

@app.callback(
    Output('price-chart','figure'),
    [Input('button','n_clicks')],
    [State('ticker-dd','value'),
     State('date','start_date'),
     State('date','end_date')])
    
def update_graph(n_clicks,value,start_date,end_date):
    i = 0
    fig = go.Figure()
    if n_clicks is None:
        raise PreventUpdate
    else:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        for i in range(len(value)):     
            df = get_historical_data(value[i], start_date, end_date, token="xxxxxxxxx", output_format='pandas')
            df.index = pd.to_datetime(df.index)
            fig.add_trace(go.Scatter(
                x = df.index,
                y = df['close'],
                mode = 'lines',
                name = value[i]
                ))
    
    fig.update_layout(xaxis_range=[start_date,end_date],title='{} Closing Price'.format(', '.join(value)))
    fig.show()
    return fig

if __name__ == '__main__':
    app.run_server(port=8080)